# Progetto - Spotify API

### Progetto di Francesco Petrosino X81000533


## Introduzione

testo

## Come funziona

testo

### Installare le librerie richieste

Esegui questo comando nel tuo terminale: `py -m pip install --upgrade -r .\requirements.txt`

In [1]:
# Imports libraries

import time
import pandas
import sklearn
import json
import spotipy

from group import Group
from spotipy.oauth2 import SpotifyOAuth
from user import User

import main

In [2]:
# Sets variables

# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
# redirect_uri = "YOUR_REDIRECT_URI"
client_id = "b48898f561ab4533b7396439e6610ff4"
client_secret = "bbf061893e8f4ff8924eed41c39c6a90"
redirect_uri = "http://localhost:8080"

scope = [
	"user-library-read",
	"user-top-read",
	"user-follow-read",
	"user-read-private",
	"user-read-recently-played",
	"user-read-playback-state",
	"user-modify-playback-state",
	"user-read-playback-position",
	"streaming",
	"app-remote-control",
	"playlist-read-private",
	"playlist-modify-private",
	"playlist-modify-public",
	"playlist-read-collaborative"
]

# Authenticates

auth_manager = SpotifyOAuth(
	client_id=client_id,
	client_secret=client_secret,
	redirect_uri=redirect_uri,
	scope=scope
)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
# Defines a list of users

# Static list

group = Group()

id_list = [
	'21ktslj6lkgx6wswmfwu5cwuq',
	'rcin690qmgjd32d088agw6xyr',
	'1171706601',
	'6ua7zipvurza24psy4h2rvumy',
	'21vfi35alpob2afo2vtoybthi',
	'392q67hc38jnsgjg2hqrjyuhc',
	'11135404841'
]

print("Creo il gruppo...")

len = len(id_list)

for i, id in enumerate(id_list):
	result = sp.user(id)

	user = User(result)
	group.append(user)

	if i+1 < len:
		time.sleep(3)

group.print()

Creo il gruppo...
Il gruppo è composto da: 
Francesco Petrosino
Ale
Philip Giovanni Emmer
Renato
Elio Pampolini
Scalòppina
Aldo Fiorito


In [4]:
print("Prendo le playlists dal server...")

len = group.count()

for i in range(len):
	user = group.get_from_list(i)

	print(user.display_name)

	results = sp.user_playlists(user=user.user_id, limit=10)
	user.set_following_playlists(results['items'])

	if i+1 < len: 
		time.sleep(5)

Prendo le playlists dal server...
Francesco Petrosino
Ale
Philip Giovanni Emmer
Renato
Elio Pampolini
Scalòppina
Aldo Fiorito


In [5]:
print("Creo un dataframe con le playlists...")

len = group.count()

for i in range(len):
	# Prendo l'utente corrente
	user = group.get_from_list(i)

	# Creo una lista temporanea delle playlist dell'utente corrente
	playlist_list = list(map(lambda playlist: {
		'user_id': user.user_id,
		'user_display_name': user.display_name,
		'playlist_id': playlist.get('id'),
		'playlist_name': playlist.get('name'),
		'playlist_description': playlist.get('description'),
		'playlist_uri': playlist.get('uri'),
		'playlist_public': playlist.get('public'),
		'playlist_total_tracks': playlist.get('tracks', {}).get('total')
	}, user.get_following_playlists()))

	# Creo un dataframe con la playlist dell'utente corrente
	playlist_user_dataframe = pandas.DataFrame(playlist_list)

	# Concateno con il dataframe finale
	group.append_to_playlist_dataframe(playlist_user_dataframe)


Creo un dataframe con le playlists...


s:\Coding\ProgettoSMM\Progetto2\group.py:35: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:35: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:35: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_dataframe = pandas.concat([self.playlist_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:35: Futu

In [7]:
print('Per ogni playlist cerco le relative tracks')

for index, playlist in group.get_playlist_dataframe().iterrows():
    print(playlist.get('playlist_name'))

    # Prendo le tracks in una playlist
    results = sp.playlist_items(playlist_id=playlist.get('playlist_id'), limit=100)
    
    items_raw = json.dumps(results['items'])
    items_json = json.loads(items_raw)
    items_list = list(items_json)
    
    # Creo una lista temporanea delle playlist dell'utente corrente
    playlist_tracks_list = list(map(lambda track: {
        'user_id': playlist.get('user_id'),
        'user_display_name': playlist.get('user_display_name'),
        'playlist_id': playlist.get('playlist_id'),
        'playlist_name': playlist.get('playlist_name'),
        'album_id': track.get('track', {}).get('album', {}).get('id'),
        'album_name': track.get('track', {}).get('album', {}).get('name'),
        'album_release_date': track.get('track', {}).get('album', {}).get('release_date'),
        'album_total_tracks': track.get('track', {}).get('album', {}).get('total_tracks'),
        'artist_id': track.get('track', {}).get('artists', {})[0].get('id'),
        'artist_name': track.get('track', {}).get('artists', {})[0].get('name'),
        'track_disc_number': track.get('track', {}).get('disc_number'),
        'track_duration_ms': track.get('track', {}).get('duration_ms'),
        'track_explicit': track.get('track', {}).get('explicit'),
        'track_id': track.get('track', {}).get('id'),
        'track_name': track.get('track', {}).get('name'),
        'track_popularity': track.get('track', {}).get('popularity')
    }, items_list))

    # Creo un dataframe con la playlist dell'utente corrente
    playlist_tracks_dataframe = pandas.DataFrame(playlist_tracks_list)

    # Concateno con il dataframe finale
    group.append_to_playlist_tracks_dataframe(playlist_tracks_dataframe)


Per ogni playlist cerco le relative tracks
Car
Guitar


s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


2021


s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


- MUSIC (R) 


...

s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


Chill and Memes


s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)
s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


Acoustic Mood
🩹


s:\Coding\ProgettoSMM\Progetto2\group.py:48: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self.playlist_tracks_dataframe = pandas.concat([self.playlist_tracks_dataframe, dataframe], ignore_index=True)


In [ ]:
main.export_to_csv('dist/playlists.csv', group.get_playlist_dataframe())

In [9]:
main.export_to_csv('dist/tracks.csv', group.get_playlist_tracks_dataframe())

In [4]:
group.playlist_dataframe = main.import_from_csv('dist/playlists.csv')
group.playlist_tracks_dataframe = main.import_from_csv('dist/tracks.csv')

In [5]:
print(group.get_playlist_dataframe().to_csv())

,Unnamed: 0,user_id,user_display_name,playlist_id,playlist_name,playlist_description,playlist_uri,playlist_public,playlist_total_tracks
0,0,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,1axIU0KXgXndDVsOO2Mzfy,Car,,spotify:playlist:1axIU0KXgXndDVsOO2Mzfy,True,28
1,1,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,4CiKz5cbyJUSj6bIAHZdgl,Guitar,,spotify:playlist:4CiKz5cbyJUSj6bIAHZdgl,True,34
2,2,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,136beH7rbmEUpqVr3plQpz,2021,,spotify:playlist:136beH7rbmEUpqVr3plQpz,True,14
3,3,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,4R5YfdibIeQXPOtuKTE9E1,- MUSIC (R) ,,spotify:playlist:4R5YfdibIeQXPOtuKTE9E1,True,48
4,4,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,4NnDxZXf0xHg2nUEiXcqhE,2020 Look up,,spotify:playlist:4NnDxZXf0xHg2nUEiXcqhE,True,18
5,5,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,7zgEXqu8hWFGZT69B6Y184,Relax,,spotify:playlist:7zgEXqu8hWFGZT69B6Y184,True,2
6,6,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,,spotify:playlist:

In [6]:
print(group.get_playlist_tracks_dataframe().to_csv())

,Unnamed: 0,user_id,user_display_name,playlist_id,playlist_name,album_id,album_name,album_release_date,album_total_tracks,artist_id,artist_name,track_disc_number,track_duration_ms,track_explicit,track_id,track_name,track_popularity
0,0,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,1axIU0KXgXndDVsOO2Mzfy,Car,2vGU0DOcfsDee0euvhl1iZ,B4 The Storm,2020-08-28,17,6MPCFvOQv5cIGfw3jODMF0,Internet Money,1,195428,True,7hxHWCCAIIxFLCzvDgnQHX,"Lemonade (feat. Gunna, Don Toliver & NAV)",78
1,1,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,1axIU0KXgXndDVsOO2Mzfy,Car,4g1ZRSobMefqF6nelkgibi,Hollywood's Bleeding,2019-09-06,17,246dkjvS1zLTtiykXe5h60,Post Malone,1,149546,True,7xQAfvXzm3AkraOtGPWIZg,Wow.,83
2,2,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,1axIU0KXgXndDVsOO2Mzfy,Car,1BayCxyyUYlm3x0Ys1z4Fl,Notti Brave,2018-05-11,15,29nrwultxQ8jENVmXoyMqK,Carl Brave,1,185840,False,4qB5WfP7AzwPzU4OddDm4b,Chapeau (feat. Frah Quintale),54
3,3,21ktslj6lkgx6wswmfwu5cwuq,Francesco Petrosino,1axIU0KXgXndDVsOO2Mzfy,

## Analisi dei dati

testo


### Collaborative filtering

Tra i **sistemi di raccomandazione** è noto il metodo **Collaborative filtering** per formulare raccomandazioni agli utenti in base al loro comportamento passato e al comportamento di utenti simili.<br>

Il sistema utilizza per identificare modelli e somiglianze tra loro, quindi utilizza questi modelli per formulare raccomandazioni ai nuoviutenti.<br>
<br>
Ci sono due tipi principali di Collaborative filtering:

- **User-based CF**: Questo metodo esamina il comportamento passato di utenti simili per formulare raccomandazioni a un nuovo utente. Ad esempio, se un utente A ha preferenze simili a quelle dell'utente B e l'utente B ha valutato altamente un determinato elemento, il sistema consiglierebbe quell'elemento all'utente A.

- **Item-based CF**: Questo metodo esamina il comportamento passato degli utenti rispetto a elementi specifici. Ad esempio, se un utente A ha valutato l'elemento X altamente e l'elemento Y altamente, e un altro utente B ha anche valutato l'elemento X altamente, il sistema consiglierebbe l'elemento Y all'utente B.

<br>
Per implementare un sistema CF, il primo passo è raccogliere dati sul comportamento passato degli utenti. Questi dati vengono quindi utilizzati per creare una matrice di elementi utente che rappresenta le valutazioni che gli utenti hanno dato agli elementi. Successivamente, il sistema utilizza una misura di somiglianza (come la somiglianza coseno) per identificare utenti o elementi simili. Infine, il sistema utilizza gli utenti o gli elementi simili identificati per formulare raccomandazioni ai nuovi utenti.<br>
<br>
È importante notare che il filtro collaborativo è una tecnica basata sulla memoria, significa che l'algoritmo deve avere accesso a tutte le interazioni utente-elemento per generare raccomandazioni, ecco perché non è adatto per set di dati di grandi dimensioni. Inoltre, può essere influenzato dal problema del cold-start, significa che l'algoritmo non può raccomandare elementi ai nuovi utenti che non hanno ancora interagito con il sistema.<br>
<br>
Inoltre, il filtro collaborativo può anche essere classificato in base alla disponibilità di feedback:

- **Explicit feedback**: Questo tipo di feedback si ottiene attraverso valutazioni esplicite, ad esempio, un utente che valuta un film da 1 a 5 stelle.
- **Implicit feedback**: Questo tipo di feedback si ottiene attraverso azioni implicite, ad esempio un utente che guarda un film o ascolta un brano. Il feedback implicito può essere più difficile da lavorare rispetto al feedback esplicito, in quanto può essere rumoroso e scarso.

<br>
Infine, è anche importante ricordare che i sistemi CF sono spesso combinati con altre tecniche, come il Content-based filtering, per superare i limiti e migliorare la qualità delle raccomandazioni.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# read the user-item interactions dataset
df = pd.read_csv('user_item_interactions.csv')

# create a pivot table of the interactions
user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='rating')

# compute the cosine similarity matrix
cosine_sim = cosine_similarity(user_item_matrix)

# create a mapping of user_id to index in the similarity matrix
user_index_map = {user_id: index for index, user_id in enumerate(user_item_matrix.index)}

# function to get the top k similar users
def get_similar_users(user_id, k=5):
    user_index = user_index_map[user_id]
    similar_users = list(enumerate(cosine_sim[user_index]))
    similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)
    similar_users = similar_users[1:k+1]
    return similar_users

# function to make recommendations to a user
def recommend_items(user_id, k=5):
    similar_users = get_similar_users(user_id, k)
    similar_users_index = [index for index, similarity in similar_users]
    similar_users_ratings = user_item_matrix.iloc[similar_users_index]
    recommended_items = similar_users_ratings.mean().sort_values(ascending=False)
    return recommended_items

# test the recommendation function
user_id = 123
print(recommend_items(user_id))

ModuleNotFoundError: No module named 'sklearn'

In [8]:
import collaborative_filtering
from sklearn.metrics.pairwise import cosine_similarity

# read the user-item interactions dataset
df = pandas.read_csv('dist/tracks.csv', sep=";")

# create a pivot table of the interactions
user_item_matrix = df.pivot_table(columns='track_id', values='track_popularity')

# compute the cosine similarity matrix
cosine_sim = cosine_similarity(user_item_matrix)

# create a mapping of user_id to index in the similarity matrix
user_index_map = {user_id: index for index, user_id in enumerate(user_item_matrix.index)}

# test the recommendation function
user_id = '21ktslj6lkgx6wswmfwu5cwuq'
print(collaborative_filtering.recommend_items(user_id, user_item_matrix, cosine_sim, k=5))

KeyError: '21ktslj6lkgx6wswmfwu5cwuq'

In this example, the **pandas** library is used to read the user-item interactions dataset and create a pivot table. The **sklearn** library is used to compute the cosine similarity between users.<br><br>
The function **get_similar_users** takes a user_id and the number of similar users to return as input and returns a list of similar users based on the cosine similarity.<br>
The function **recommend_items** takes a user_id and the number of similar users to consider as input and returns a list of recommended items based on the mean ratings of the similar users.<br><br>
You can also add or use other similarity measures like Jaccard Similarity, Pearson Correlation, etc
It's important to note that this is a basic example, in practice, you might need to handle missing values, normalize the ratings, and use other techniques like cross-validation to evaluate the model performance.

### Content-based filtering

Il **Content-based filtering** è un metodo del sistema di raccomandazione per formulare raccomandazioni agli utenti in base alle caratteristiche o agli attributi degli elementi a cui gli utenti hanno mostrato interesse. Il sistema utilizza questi attributi per identificare elementi simili e quindi formula raccomandazioni agli utenti in base alla somiglianza.<br>
<br>
L'idea generale alla base di questo metodo è che se a un utente piace un elemento, gli piaceranno anche elementi simili. Per implementare un sistema di filtraggio basato sul contenuto, il primo passo è estrarre feature o attributi dagli elementi. Queste caratteristiche possono essere tutto ciò che caratterizza l'oggetto, come il suo genere, regista, attori, ecc. Successivamente, viene utilizzata una misura di somiglianza, come la somiglianza del coseno, per confrontare i vettori delle caratteristiche degli elementi per identificare elementi simili. Infine, il sistema utilizza gli elementi simili identificati per formulare raccomandazioni agli utenti.<br>
<br>
Questo modello può essere diviso in due tipi principali:
- **Item-content based**: Questo metodo utilizza le caratteristiche dell'elemento per consigliare elementi simili all'utente. Ad esempio, se a un utente piace un film che è classificato come "commedia" e ha "Adam Sandler" come attore, il sistema consiglierà altri film che sono anche classificati come "commedia" e hanno "Adam Sandler" come attore.
- **User-content based**: Questo metodo utilizza le funzionalità dell'utente per consigliare elementi simili alle preferenze dell'utente. Ad esempio, se a un utente piacciono i generi "commedia" e "azione", il sistema consiglierà elementi che provengono da questi generi.

<br>
È importante notare che il filtraggio basato sui contenuti è limitato dalla qualità e dalla quantità delle caratteristiche dell'articolo, se le caratteristiche non sono informative o pertinenti, le raccomandazioni non saranno accurate. Inoltre, non è adatto a raccomandare nuovi articoli, deve avere le caratteristiche dell'articolo nel database. La combinazione del filtraggio basato sui contenuti con altre tecniche come il Collaborative filtering può superare questi limiti.<br>
<br>

Il metodo **Content-based filtering** prevede in genere i seguenti passaggi:
- **Raccogliere e preelaborare le caratteristiche dell'oggetto**: il primo passo è raccogliere le caratteristiche degli oggetti, come il genere, il regista, gli attori, ecc. e preelaborarle per l'analisi. Questo passaggio può includere la pulizia dei dati, la rimozione dei valori mancanti e la conversione di feature categoriche in valori numerici.
- **Creare un vettore di feature per ogni elemento**: una volta che le feature sono preelaborate, il passo successivo consiste nel creare un vettore di feature per ogni elemento. Questo vettore rappresenta le caratteristiche dell'elemento in forma numerica e viene utilizzato per confrontare la somiglianza tra gli elementi.
- **Calcolare la somiglianza tra gli elementi**: dopo aver creato i vettori di feature, il sistema utilizza una misura di somiglianza come la somiglianza del coseno, la somiglianza di Jaccard o la distanza euclidea per calcolare la somiglianza tra gli elementi.
- **Fare raccomandazioni**: una volta calcolata la somiglianza tra gli elementi, il sistema può formulare raccomandazioni agli utenti in base alla somiglianza. Ad esempio, se un utente ha mostrato interesse per un elemento specifico, il sistema può raccomandare altri elementi simili all'utente.
- **Rivalutare e aggiornare il modello**: il passo finale è rivalutare le prestazioni del modello e aggiornarlo se necessario. Questo passaggio può includere l'aggiunta di nuovi elementi, la rimozione di elementi irrilevanti e la regolazione del peso delle feature.

<br>
Vale la pena notare che il metodo di filtraggio basato sui contenuti può essere migliorato includendo informazioni aggiuntive, come le informazioni demografiche dell'utente, o integrandolo con altre tecniche di raccomandazione, come il Collaborative filtering.<br>
<br>
Inoltre, mentre il Content-based filtering è adatto per raccomandare elementi agli utenti con una storia di interazioni, non è bravo a raccomandare elementi a nuovi utenti senza alcuna cronologia, a questo scopo, può essere utilizzata una tecnica di filtraggio ibrido che combina i punti di forza sia del filtraggio basato sui contenuti che del filtraggio collaborativo.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import itertools

s1 = '21ktslj6lkgx6wswmfwu5cwuq'
s2 = 'rcin690qmgjd32d088agw6xyr'
s3 = '1171706601'
s4 = '21ktslj6lkgx6wswmfwu5cwuq'

vectorizer = CountVectorizer().fit_transform([s1, s2, s3, s4])
vectors = vectorizer.toarray().tolist()
v1d = vectors[0]

print(list(itertools.chain.from_iterable(vectors)))
print('======')

print(vectors, type(vectors))
print('======')

similarity = cosine_similarity(vectors)

print(similarity)

[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]
[[0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0]] <class 'list'>
[[1. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 1.]]


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy

# define your data
data = [[7, 8], [1, 2], [7, 8], [10, 11]]

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(data)

print(similarity_matrix)
print('===================')

# define number of similar items you want to get
n = 3

# sort the similarity matrix in descending order
sorted_matrix = numpy.argsort(-similarity_matrix)

# get the indices of the n most similar items
most_similar = sorted_matrix[:, -n:]
most_similar2 = numpy.argsort(similarity_matrix)[:,-n-1:-1]

# get the corresponding items based on the indices obtained
similar_items = [data[i] for i in most_similar[0]]
similar_items2 = [data[i] for i in most_similar2[0]]

print(similar_items)
print('===================')
print(similar_items2)

[[1.         0.96761727 1.         0.99981979]
 [0.96761727 1.         0.96761727 0.96265094]
 [1.         0.96761727 1.         0.99981979]
 [0.99981979 0.96265094 0.99981979 1.        ]]
[[7, 8], [10, 11], [1, 2]]
[[1, 2], [10, 11], [7, 8]]


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

# read the item dataset
df = pd.read_csv('items.csv')

# create a feature vector for each item
df['feature_vector'] = df[['feature1', 'feature2', 'feature3']].apply(lambda row: [row['feature1'], row['feature2'], row['feature3']], axis=1)

# compute the cosine similarity matrix
cosine_sim = cosine_similarity(df['feature_vector'])

# function to get the top k similar items
def get_similar_items(item_id, k=5):
    item_index = df[df['item_id'] == item_id].index[0]
    similar_items = list(enumerate(cosine_sim[item_index]))
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
    similar_items = similar_items[1:k+1]
    return similar_items

# function to make recommendations to a user
def recommend_items(user_id, k=5):
    user_items = df[df['user_id'] == user_id]['item_id']
    recommended_items = []
    for item_id in user_items:
        similar_items = get_similar_items(item_id, k)
        recommended_items += [item_index for item_index, similarity in similar_items]
    recommended_items = df.iloc[recommended_items]
    return recommended_items

# test the recommendation function
user_id = 123
print(recommend_items(user_id))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

ValueError: setting an array element with a sequence.

In [26]:
import numpy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# read the item dataset
df = pandas.read_csv('dist/tracks.csv', sep=';')

# create a feature vector for each item
df['user_id_cod'] = CountVectorizer().fit_transform(df['user_id']).toarray().tolist()
df['playlist_id_cod'] = CountVectorizer().fit_transform(df['playlist_id']).toarray().tolist()
df['track_id_cod'] = CountVectorizer().fit_transform(df['track_id']).toarray().tolist()

# dataframe tmp con i coseni
df_cos = pandas.DataFrame({
    'user_id_cod': df['user_id_cod'],
    'playlist_id_cod': df['playlist_id_cod'],
    'track_id_cod': df['track_id_cod']
})

# define number of similar items you want to get
n = df.shape[0]

df_list = numpy.array(df_cos['track_id_cod'])

# sort the similarity matrix in descending order
# sorted_matrix = numpy.argsort(-df_list.tolist())
sorted_matrix = numpy.argsort(df_list.tolist())[:,-n-1:-1]

# get the indices of the n most similar items
most_similar = sorted_matrix[:, -n:]

# get the corresponding items based on the indices obtained
similar_items = [df['track_id_cod'][i] for i in most_similar[0]]
print(len(similar_items))
# print(df['user_id_cod'] - df_list['user_id_cod'])
# for i in similar_items:
#     print(type(i), i)

# compute the cosine similarity matrix
# cosine_sim = cosine_similarity(df['track_id_cod'].tolist())
# print(len(cosine_sim))

# # function to get the top k similar items
# def get_similar_items(item_id, y_column_name, k=5):
#     item_index = df[df[y_column_name] == item_id].index[0]
#     similar_items = list(enumerate(cosine_sim[item_index]))
#     similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
#     similar_items = similar_items[1:k+1]
#     return similar_items

# # function to make recommendations to a user
# def recommend_items(user_id, y_column_name, k=5):
#     user_items = df[df['user_id'] == user_id][y_column_name]
#     recommended_items = []
#     for item_id in user_items:
#         similar_items = get_similar_items(item_id, y_column_name, k)
#         recommended_items += [item_index for item_index, similarity in similar_items]
#     recommended_items = df.iloc[recommended_items]
#     return recommended_items

# # test the recommendation function
# user_id = 123
# # print(recommend_items(user_id, 'track_id_cod'))
# print(get_similar_items('3BVhZHB2DDDwwrjOQtLfcY', 'track_id_cod'))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In this example, the **pandas** library is used to read the item dataset and create a feature vector for each item. The **sklearn** library is used to compute the cosine similarity between items.
The function **get_similar_items** takes an item_id and the number of similar items to return as input and returns a list of similar items based on the cosine similarity.
The function **recommend_items** takes a user_id and the number of similar items to consider as input and returns a list of recommended items based on the user's items.
You can also add or use other similarity measures like Jaccard Similarity, Pearson Correlation, etc..<br>
<br>
It's important to note that this is a basic example, in practice, you might need to handle missing values, normalize the ratings, and use other techniques like cross-validation to evaluate the model performance.
Also, this implementation is item-content based filtering, you can also implement user-content based filtering by using the user's characteristics in the feature vector instead of the items.

In [21]:
print(type(similar_items[0]))
print(type(df['track_id_cod']))
# df[df['track_id_cod'] == similar_items[0]]

<class 'list'>
<class 'pandas.core.series.Series'>


In [22]:
for index, value in similar_items.iteritems():
    if df.loc[index, 'track_id_cod'] == value:
        print(value, "matches the value in the dataframe at index", index)
    # print(l)

AttributeError: 'list' object has no attribute 'iteritems'

In [25]:
for value1, value2 in zip(similar_items, df['track_id_cod']):
    if value1 == value2:
        print(df[df['track_id_cod'][:] == value1])

ValueError: ('Lengths must match to compare', (1108,), (1069,))

In [26]:
len(df)

1108

In [19]:
df[df['user_id'] == 'rcin690qmgjd32d088agw6xyr']

,Unnamed: 0,user_id,user_display_name,playlist_id,playlist_name,album_id,album_name,album_release_date,album_total_tracks,artist_id,artist_name,track_disc_number,track_duration_ms,track_explicit,track_id,track_name,track_popularity,user_id_cod,playlist_id_cod,track_id_cod
144,144,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,43dl8hP5uNN9s6YeeJA5fv,DÁKITI,2020-10-30,1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,1,205090,True,47EiUVwUp4C9fGccaPuUCS,DÁKITI,83,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
145,145,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,3fxzSn0ObgCjLadyR53ohN,Vete,2019-11-21,1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,1,192024,True,5DxXgozhkPLgrbKFY91w0c,Vete,79,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
146,146,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,6VSOIs13DaSG2IPilNviX5,Yonaguni,2021-06-04,1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,1,206710,True,2JPLbjOn0wPCngEot2STUS,Yonaguni,89,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
147,147,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,06S3Qjh4QWLtn6c7CVhYh7,Callaita,2019-05-31,1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,1,250533,True,2TH65lNHgvLxCKXM3apjxI,Callaita,80,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
148,148,rcin690qmgjd32d088agw6xyr,Ale,251IXQ6UPt8hQE1xDg73my,el latino,6ylFfzx32ICw4L1A7YWNLN,OASIS,2019-06-28,8,1vyhD5VmyZ7KMfW5gqLgo5,J Balvin,1,242573,False,0fea68AdmYNygeTGI4RC18,LA CANCIÓN,89,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,248,rcin690qmgjd32d088agw6xyr,Ale,3LuKO6vbZDcFSNihsLfeui,Alan,4blCxKnK1sGo9D45trrqjt,Alone,2016-12-02,3,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,1,161200,False,0JiVRyTJcJnmlwCZ854K4p,Alone,64,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
249,249,rcin690qmgjd32d088agw6xyr,Ale,3LuKO6vbZDcFSNihsLfeui,Alan,4yKeyQkqI89bpEQRmgUWg3,Ignite (feat. SEUNGRI),2018-06-07,1,6pWcSL9wSJZQ9ne0TnhdWr,K-391,1,210288,False,0UMLejbZXf7YVgnfxURvVr,Ignite (feat. SEUNGRI),0,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
250,250,rcin690qmgjd32d088agw6xyr,Ale,3LuKO6vbZDcFSNihsLfeui,Alan,4hlAdqONoJhkjf8u9XMjQr,All Falls Down (feat. Juliander),2017-10-27,1,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,1,199111,False,5yFSF6qQA1rcLsQRnBsZgb,All Falls Down (feat. Juliander),65,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
251,251,rcin690qmgjd32d088agw6xyr,Ale,3LuKO6vbZDcFSNihsLfeui,Alan,5HMjpBO0v78ayq5lreAyDd,Faded,2015-12-04,4,7vk5e3vY1uw9plTHJAMwjN,Alan Walker,1,212626,False,7gHs73wELdeycvS48JfIos,Faded,73,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
